In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

drev = pd.read_csv("../../web/exports/drev.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Siret Opérateur': 'str', 'Code postal Opérateur': 'str'}, low_memory=False)
etablissements = pd.read_csv("../../web/exports/etablissements.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Login': 'str', 'Identifiant etablissement': 'str'}, index_col=False, low_memory=False)
dr = pd.read_csv("../../web/exports/dr.csv", encoding="iso8859_15", delimiter=";",decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Valeur': 'float64'}, low_memory=False)
societe = pd.read_csv("../../web/exports/societe.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Téléphone' :'str', 'Téléphone portable': 'str'}, index_col=False, low_memory=False)

In [ ]:
#def createCSVCOTIDREVCRUByCampagne(campagne,drev,etablissements,dr,societe):
campagne = "2020"   
drev = drev.query("Campagne == @campagne");

drev = drev.groupby(['Identifiant','Appellation','CVI Opérateur']).sum()

drev = drev.reset_index(level=['Identifiant', 'Appellation','CVI Opérateur'])

dr = dr.query("Campagne == @campagne");

dr = dr.query("Appellation != 'CDR' and Appellation != 'CVG'")

dr["Bailleur PPM"] = dr['Bailleur PPM'].fillna("")

dr = pd.pivot_table(dr, values= 'Valeur', index=['Identifiant',"Appellation",'Bailleur PPM','CVI'],columns=['Categorie'], aggfunc=np.sum)

dr = dr.reset_index(level=['Bailleur PPM','CVI'])

drev_cru = drev.query("Appellation != 'CDR' and Appellation != 'CVG'")

drev_cru_with_dr = pd.merge(drev_cru, dr, how='outer',left_on=["Identifiant",'Appellation'], right_on=["Identifiant","Appellation"],suffixes=("", " societe"))

drev_cru_with_dr['AOCID'] = drev_cru_with_dr['Appellation']+'-'+drev_cru_with_dr['Identifiant']

drev_cru_with_dr['AOCIDBAIL'] = drev_cru_with_dr['Bailleur PPM']

drev_cru_with_dr['AOC'] = drev_cru_with_dr['Appellation']

drev_cru_with_dr['DREV_SURF'] = drev_cru_with_dr['Superficie revendiqué']

drev_cru_with_dr['DREV_VOL'] = drev_cru_with_dr['Volume revendiqué net total']

drev_cru_with_dr['DR_SURF'] = drev_cru_with_dr[[' Superficie de récolte','Superficie de recolte','Superficie de récolte','Superificie de récolte originale']].max(axis=1)

drev_cru_with_dr['DR_REC_TOTALE'] = drev_cru_with_dr[['Recolte totale','Récolte totale','Recolte totale',' Récolte totale']].max(axis=1)

drev_cru_with_dr['DR_VF RAISIN'] = drev_cru_with_dr[['Récolte vendue sous forme de raisins. Volume de vins obtenu','Recolte vendue sous forme de raisins. Volume de vins obtenu',' Récolte vendue sous forme de raisins. Volume de vins obtenu']].max(axis=1)

drev_cru_with_dr['DR_VF MOUT'] = drev_cru_with_dr[['Recolte vendue sous forme de mouts. Volume de mout obtenu','Récolte vendue sous forme de moûts. Volume de moût obtenu']].max(axis=1)

drev_cru_with_dr['DR_CAVE COOP'] = drev_cru_with_dr[[" Récolte apportée en cave coopérative par l'adhérent. Volume obtenu",'Recolte apportee en cave cooperative par l?adherent. Volume obtenu',"Récolte apportée en cave coopérative par l'adhérent. Volume obtenu"]].max(axis=1)

drev_cru_with_dr['DR_CAVE PART'] = drev_cru_with_dr[[' Récolte en cave particulière. Volume obtenu','Recolte en cave particuliere. Volume obtenu','Récolte en cave particulière. Volume obtenu']].max(axis=1)

drev_cru_with_dr['DR_VOL VINIF'] = drev_cru_with_dr[['Volume en vinification',' Volume en vinification']].max(axis=1)

drev_cru_with_dr['DR_VOL AOC'] = drev_cru_with_dr[['Vol. de vin avec AO/IGP avec/sans cépage dans la limite du rdt autorisé',' Vol. de vin avec AO/IGP avec/sans cépage dans la limite du rdt autorisé','Vol. de vin avec AO/IGP avec/sans cepage dans la limite du rdt autorise']].max(axis=1)

drev_cru_with_dr['DR_VOL UI'] = drev_cru_with_dr[[' Vol. vin dépassement du rdt autorisé en AOP à livrer aux usages industriels','Vol. vin depassement du rdt autorise en AOP a livrer aux usages industriels','Vol. vin dépassement du rdt autorisé en AOP à livrer aux usages industriels']].max(axis=1)

drev_cru_with_dr['AOC'] = drev_cru_with_dr['Appellation']

drev_cru_with_dr['ID_BAIL'] = ""

drev_cru_with_dr["AOCIDBAIL"] = drev_cru_with_dr['Bailleur PPM'].fillna("")

drev_cru_with_dr["CVI_OPERATEUR"] = drev_cru_with_dr['CVI Opérateur'].fillna("")


drev_cru_with_dr.loc[drev_cru_with_dr.AOCIDBAIL != "" , 'DREV_SURF'] = np.nan
drev_cru_with_dr.loc[drev_cru_with_dr.AOCIDBAIL != "" , 'DREV_VOL'] = np.nan

drev_cru_with_dr.loc[drev_cru_with_dr.CVI_OPERATEUR == "" , 'CVI_OPERATEUR'] = drev_cru_with_dr["CVI"]


etablissements = pd.merge(societe, etablissements, how='inner',left_on="Identifiant", right_on="Login",suffixes=("", " etablissement"))


drev_cru_with_dr = pd.merge(drev_cru_with_dr,etablissements, left_on ='CVI_OPERATEUR', right_on = 'CVI',suffixes=("", " etablissement 2"))


drev_cru_with_dr['ID'] = drev_cru_with_dr['Identifiant etablissement 2']
drev_cru_with_dr.loc[drev_cru_with_dr.AOCIDBAIL != "" , 'ID_BAIL'] = drev_cru_with_dr.ID
drev_cru_with_dr.loc[drev_cru_with_dr.AOCIDBAIL != "" , 'ID'] = drev_cru_with_dr.AOCIDBAIL

drev_cru_with_dr = drev_cru_with_dr.rename(columns={'Identifiant etablissement': "ID_ETABLISSEMENT"})


drev_cru_with_dr = drev_cru_with_dr[['AOCID', 'AOCIDBAIL','Appellation', 'DREV_SURF' , 'DREV_VOL', 'DR_SURF', 'DR_REC_TOTALE','DR_VF RAISIN','DR_VF MOUT',"DR_CAVE COOP",'DR_CAVE PART','DR_VOL VINIF','DR_VOL AOC','DR_VOL UI','ID','ID_BAIL','CVI_OPERATEUR','Raison sociale','Adresse','Adresse 2','Adresse 3','Code postal','Commune','Téléphone','Téléphone portable','Email']]

drev_cru_with_dr.to_csv('../../web/exports/COTIDREV'+campagne+'CRU.csv', encoding="iso8859_15", sep=";", decimal=",")

In [ ]:
#createCSVCOTIDREVCRUByCampagne('2020',drev,etablissements,dr,societe)